In [2]:

import generator
from generator.main import read_data, format_data, generate_text, create_idx_to_words,  create_model, \
    read_model_weights

from generator.main import get_parameters

In [3]:
parameters = get_parameters()
parameters

Parameters(model_name='model', nrows=200000, max_features=3500, max_len=20, epochs=3, start='÷', end='■')

Data Processing

In [4]:
raw_data = read_data(nrows=parameters.nrows)
raw_data


,headline
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
...,...
199995,glory on top of knights
199996,gm canola contamination report released
199997,governments work towards charlestown masterplan
199998,govt accused of using bullying tactics to force


In [5]:
X, Y, tokenizer, text = format_data(raw_data, parameters.max_features, parameters.max_len, parameters.start, parameters.end)
X, Y, tokenizer, text

(array([[   0,    0,    0, ...,    0,    1,   37],
        [   0,    0,    0, ...,    1,   37,  187],
        [   0,    0,    0, ...,   37,  187, 1250],
        ...,
        [   0,    0,    0, ...,   10,   18,   36],
        [   0,    0,    0, ...,   18,   36, 1049],
        [   0,    0,    0, ...,   36, 1049,  697]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32),
 0         ÷ aba decides against community broadcasting l...
 1         ÷ act fire witnesses must be aware of defamati...
 2         ÷ a g calls for infrastructure protection summ...
 3              ÷ air nz staff in aust strike for pay rise ■
 4         ÷ air nz strike to affect australian travellers ■
                                 ...                        
 199995                          ÷ glory on top o

In [6]:
tokenizer.word_index['community'], tokenizer.word_index[parameters.start], tokenizer.word_index[parameters.end]

(187, 1, 2)

Model Building

In [7]:
#build_model(parameters.model_name, parameters.epochs, parameters.max_features, X, Y)
model = create_model(parameters.max_features)
read_model_weights(parameters.model_name, model)
model.evaluate(X,Y)

32452/32452 [==============================] - 250s 8ms/step - loss: 4.8743 - accuracy: 0.2677


[4.867789268493652, 0.26980096101760864]

Text Generation

In [8]:
idx_to_words= create_idx_to_words(tokenizer)



In [9]:
text = generate_text(parameters.start, model, idx_to_words, tokenizer, parameters.max_len, parameters.end, temp=0.01)
text = text[2:] # the first two elements are '÷ '
text


pred_idx:  475 pred_word:  growers
pred_idx:  3 pred_word:  to
pred_idx:  27 pred_word:  be
pred_idx:  2 pred_word:  ■
return sent:  ÷ growers to be


'growers to be'

In [10]:
text = generate_text(parameters.start, model, idx_to_words, tokenizer, parameters.max_len, parameters.end, temp=0.75)
text[2:] # the first two elements are '÷ '

pred_idx:  613 pred_word:  signs
pred_idx:  42 pred_word:  off
pred_idx:  2 pred_word:  ■
return sent:  ÷ signs off


'signs off'

In [11]:
text = generate_text('÷', model, idx_to_words, tokenizer, parameters.max_len, parameters.end, temp=1.0, min_len=5)
text = text[2:] # the first two elements are '÷ '
text


pred_idx:  1916 pred_word:  destroys
pred_idx:  1290 pred_word:  liberal
pred_idx:  449 pred_word:  poll
pred_idx:  2 pred_word:  ■
return sent:  ÷ destroys liberal poll


'destroys liberal poll'